In [0]:
query = '''SELECT owner_username as autor, slug 
FROM silver.tabnews.assunto
WHERE body IS NULL AND slug IS NOT NULL
LIMIT 50'''
resposta = spark.sql(query).collect()

In [0]:
import aiohttp
import asyncio
from typing import Optional, Dict, Any, List, Tuple
import logging

# Configura├º├úo b├ísica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

async def buscar_conteudo_TabNews_async(session: aiohttp.ClientSession, autor: str, slug: str) -> Tuple[Optional[Dict[str, Any]], str]:
    """
    Busca o conte├║do do TabNews de forma ass├¡ncrona com tratamento de erros detalhado.

    Args:
        session: A sess├úo aiohttp para reutiliza├º├úo de conex├úo.
        autor: O autor do conte├║do.
        slug: O slug do conte├║do.

    Returns:
        Uma tupla contendo:
        - Um dicion├írio com os dados do conte├║do, ou None em caso de erro.
        - Uma string com a URL da requisi├º├úo (para rastreamento e logs).
    """
    url = f'https://www.tabnews.com.br/api/v1/contents/{autor}/{slug}'
    try:
        async with session.get(url, timeout=aiohttp.ClientTimeout(total=5)) as resposta:
            #resposta.raise_for_status()  # Lan├ºa uma exce├º├úo para status HTTP de erro
            dados = await resposta.json()
            logging.info(f'Dados obtidos com sucesso para {url}')
            return dados, url
    except aiohttp.ClientError as erro:
        logging.error(f'Erro ao buscar dados da API do TabNews para {url}: {erro}')
        return None, url
    except asyncio.TimeoutError:
        logging.error(f'Timeout ao buscar dados da API do TabNews para {url}')
        return None, url
    except Exception as erro:
        logging.exception(f'Erro inesperado ao buscar dados da API do TabNews para {url}: {erro}')
        return None, url

async def buscar_conteudo_TabNews(autor: str, slug: str) -> Tuple[Optional[Dict[str, Any]], str]:
    """
    Fun├º├úo para buscar o conte├║do do TabNews, encapsulando a sess├úo aiohttp.

    Args:
        autor: O autor do conte├║do.
        slug: O slug do conte├║do.

    Returns:
        Uma tupla contendo:
        - Um dicion├írio com os dados do conte├║do, ou None em caso de erro.
        - Uma string com a URL da requisi├º├úo.
    """
    async with aiohttp.ClientSession() as session:
        return await buscar_conteudo_TabNews_async(session, autor, slug)

# Fun├º├úo para processar uma ├║nica resposta de forma ass├¡ncrona
async def processar_resposta(r) -> Dict[str, Any]:
    """
    Processa uma resposta individual, buscando o conte├║do do TabNews e tratando poss├¡veis erros.

    Args:
        r: Um objeto contendo os atributos 'autor' e 'slug'.

    Returns:
        Um dicion├írio contendo os dados da resposta, ou um dicion├írio de erro.
    """
    autor, slug = r.autor, r.slug
    dados, url = await buscar_conteudo_TabNews(autor, slug)
    if dados:
        return {'url': url, 'data': dados}
    else:
        return {'url': url, 'error': 'Falha ao obter dados'}

# Fun├º├úo principal que ser├í chamada para processar todas as respostas
async def processar_todas_respostas(resposta: List[Any]) -> List[Dict[str, Any]]:
    """
    Processa todas as respostas de forma ass├¡ncrona, utilizando asyncio.gather para paraleliza├º├úo.

    Args:
        resposta: Uma lista de objetos, cada um contendo os atributos 'autor' e 'slug'.

    Returns:
        Uma lista de dicion├írios, cada um contendo os dados da resposta ou informa├º├Áes de erro.
    """
    tarefas = [processar_resposta(r) for r in resposta]
    resultados = await asyncio.gather(*tarefas, return_exceptions=True)  # Captura exce├º├Áes individuais
    
    # Tratamento de poss├¡veis exce├º├Áes n├úo capturadas dentro de processar_resposta
    resultados_tratados = []
    for resultado in resultados:
        if isinstance(resultado, Exception):
            logging.error(f'Exce├º├úo n├úo tratada: {resultado}')
            resultados_tratados.append({'error': str(resultado)})
        else:
            resultados_tratados.append(resultado)
    
    return resultados_tratados

# Uso em um notebook Jupyter
async def main(resposta: List[Any]) -> List[Dict[str, Any]]:
    """
    Fun├º├úo principal para execu├º├úo em ambiente Jupyter Notebook.

    Args:
        resposta: Uma lista de objetos, cada um contendo os atributos 'autor' e 'slug'.

    Returns:
        Uma lista de dicion├írios contendo os resultados do processamento.
    """
    return await processar_todas_respostas(resposta)

# Supondo que 'resposta' j├í existe e cont├®m os dados
# Para executar, use:
resultados_assync = await main(resposta)
# (Em um ambiente Jupyter Notebook com suporte a asyncio)

In [0]:
import pandas as pd

def flatten_item(item):
    """
    Extrai informa├º├Áes relevantes de um item, movendo os dados aninhados para o n├¡vel superior.

    Args:
        item (dict): Um dicion├írio contendo a URL, dados (se existirem) e erros (se existirem).

    Returns:
        dict: Um dicion├írio "achatado" com a URL, dados extra├¡dos e informa├º├Áes de erro.
    """
    flat = {'url': item['url']}  # Inicializa o dicion├írio flat com a URL do item.
    flat.update(item.get('data', {}))  # Adiciona os dados do item, se existirem, ao dicion├írio flat.
    flat['error'] = item.get('error')  # Adiciona informa├º├Áes de erro, se existirem, ao dicion├írio flat.
    return flat

# Cria um DataFrame pandas a partir dos dados "achatados".
df_assync = pd.DataFrame([flatten_item(item) for item in resultados_assync])


In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col

# Carregue a tabela existente Delta
delta_table = DeltaTable.forName(spark, "silver.tabnews.assunto")
df_assync_spark = spark.createDataFrame(df_assync)
# Realize o merge usando o Delta Lake
delta_table.alias("target") \
    .merge(
        df_assync_spark.alias("updates"),
        "target.id = updates.id"
    ) \
    .whenMatchedUpdate(set={
        "url": col("updates.url"),
        "body": col("updates.body")
    }) \
    .whenNotMatchedInsert(values={
        "id": col("updates.id"),
        "url": col("updates.url"),
        "body": col("updates.body")
    }) \
    .execute()